In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import re

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [6]:
df = pd.read_csv('transformers_dataNew.csv')

In [7]:
df.dropna(subset=['content'], inplace=True)

In [8]:
df

,name,shares,wow,cares,sad,angry,haha,reactions_count,comments,content,...,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767,embeddings
0,RTL Nieuws,0,17,0,0,0,14,39,0,Medewerkers van een aquarium in Spanje aten ge...,...,13.776934,0.816872,0.773461,1.520865,1.432767,1.400064,0.703925,0.769034,1.060223,[ 2.12260270e+00 1.95452666e+00 6.88976407e-...
1,RTL Nieuws,0,1,0,0,0,10,15,0,"""Er wordt wiet gerookt in het boek, gezoend, e...",...,13.530748,1.590019,1.094324,1.871268,1.652691,1.231773,1.189072,1.068386,1.233858,[ 2.13255215e+00 1.18901312e+00 7.16963530e-...
2,RTL Nieuws,1,0,11,0,0,38,67,206,"Andy en het team van de dragbar laten zich, on...",...,13.372700,1.080609,0.964037,1.063937,1.858110,1.554261,0.934504,0.737435,1.404058,[ 2.12687016e+00 1.41113973e+00 9.98041630e-...
3,RTL Nieuws,2,4,7,19,0,0,30,26,Onder de slachtoffers waren volgens VN-mensenr...,...,13.289650,1.249961,1.254739,1.537239,1.924307,1.640161,0.366889,0.969132,0.716310,[ 2.9026854 0.780339 1.11986625 1.063458...
4,RTL Nieuws,5,12,0,0,0,21,58,284,Alleen al de afgelopen drie dagen kwamen meer ...,...,13.318510,1.263664,0.760335,1.193509,1.659033,1.697025,1.157894,1.496184,1.604272,[ 1.75874364 0.47566167 1.14991474 0.760251...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,De Telegraaf,3,0,0,0,0,47,103,169,Elke aardbewoner surft jaarlijks gemiddeld mee...,...,13.060578,1.298744,0.832913,1.100882,1.712117,0.040602,0.658332,1.102006,1.012051,[ 2.00863171 1.04274106 0.94798875 0.684139...
9996,De Telegraaf,4,0,0,0,0,17,60,26,De Nederlander en zijn team Red Bull hadden de...,...,13.239983,0.822294,0.695602,1.593539,1.619219,1.291816,0.724189,0.918574,1.635511,[ 1.101861 1.73083413 0.7798183 0.473033...
9997,De Telegraaf,2,0,0,0,23,39,71,65,„We weten weinig van de mogelijke effecten van...,...,12.788166,1.192427,0.812722,1.040017,1.249091,1.165109,0.630857,1.169717,1.338238,[ 2.35046268 1.5198431 0.71057779 1.275846...
9998,De Telegraaf,3,0,0,0,0,60,91,135,Het rommelt binnen het CDA. De Telegraaf,...,12.757520,0.974428,0.540617,1.341013,1.987473,0.697544,0.471119,0.873903,1.283651,[ 2.13793945e+00 1.02813029e+00 3.28069597e-...


In [6]:
import pandas as pd
import time
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('Indicator-Desc_DataNew.csv')
df.dropna(subset=['content'], inplace=True)
X = df['content']
y = df['entropy']

# split train dataset into train, validation and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=2018, test_size=0.3)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, random_state=2018, test_size=0.5)

#X_train, X_test, y_train, y_test = train_test_split(X , y, train_size=0.8)

# BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

# Tokenize and encode the text data
train_tokens = tokenizer.batch_encode_plus(
    X_train.values.tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

# Tokenize and encode the text data
val_tokens = tokenizer.batch_encode_plus(
    X_val.values.tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

test_tokens = tokenizer.batch_encode_plus(
    X_test.values.tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

# Convert the tokenized input to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train.values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_tokens),
    y_val.values
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test.values
))

# Prepare the model input
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Load BERT model
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow

# Freeze BERT layers
model.trainable = False

# Retrieve the BERT embeddings
embeddings = model(input_ids,attention_mask)[0]

# Perform pooling (average pooling in this case)
pooled_output = tf.keras.layers.GlobalAveragePooling1D()(embeddings)

# Intermediate dense layers with ReLU activation and dropout
hidden_units = [128, 64, 32]
dropout_rate = 0.2

for units in hidden_units:
    pooled_output = tf.keras.layers.Dense(units, activation='relu')(pooled_output)
    pooled_output = tf.keras.layers.Dropout(dropout_rate)(pooled_output)

# Dense layer for regression output
output = tf.keras.layers.Dense(1, activation='linear')(pooled_output)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model, learning rate: 1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4), loss='mean_squared_error')

model.summary()

training = model.fit(train_dataset.batch(256), epochs=5, shuffle=True, verbose=0, validation_data=val_dataset.batch(256))

# Evaluate the model on the test set
predictions = model.predict(test_dataset.batch(32))
mse = mean_squared_error(y_test.values, predictions)
print('Mean Squared Error:', mse)


Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  109137408   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

2023-06-05 10:01:34.932788: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [6982]
	 [[{{node Placeholder/_3}}]]
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2023-06-05 10:21:38.107053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [1496]
	 [[{{node Placeholder/_3}}]]
2023-06-05 12:21:08.663579: I tensorflo

47/47 [==============================] - 190s 4s/step
Mean Squared Error: 0.14353211991459983


## From https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794, Language model approach
